In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Helper Functions

In [30]:
def week_number(date):
  week=''
  weekx=date.isocalendar()[1]
  if(weekx<39):
    weekx=weekx+52
  # print(weekx-38)
  # print(weekx)
  weekx=weekx-38
  weekno = weekx%4
  if(weekno==0):
    weekno=4
  week="w"+str(weekno)
  return week

In [31]:
def week_id(date):
  week=''
  weekx=date.isocalendar()[1]
  if(weekx<39):
    weekx=weekx+52
  # print(weekx-38)
  # print(weekx)
  weekx=weekx-38
  # weekno = weekx%4
  # if(weekno==0):
  #   weekno=4
  # week="w"+str(weekx)
  return weekx

In [53]:
def week_id_valid(week):
  if week=="w1":    return 1
  elif week=="w2":    return 2
  elif week=="w3": return 3
  else: return 4

## Importing Data

In [9]:
train_data = pd.read_csv("/content/drive/MyDrive/Datastorm3.0/Semi Finals Case/data/train_data.csv", parse_dates = ['DateID'])
valid_data = pd.read_csv("/content/drive/MyDrive/Datastorm3.0/Semi Finals Case/data/validation_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Datastorm3.0/Semi Finals Case/data/test_data.csv")

promotion_data = pd.read_csv("/content/drive/MyDrive/Datastorm3.0/Semi Finals Case/data/promotion_data.csv", parse_dates = ['PromotionStartDate',	'PromotionEndDate'])

## Promotion Data

In [12]:
promotion_data.head()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
0,379249,2021-12-03,2021-12-09,10,Percentage,70
1,1066570,2021-11-19,2021-11-25,15,Percentage,45
2,124954,2021-10-29,2021-11-04,15,Percentage,210
3,35530,2021-10-22,2021-10-28,15,Percentage,60
4,1013335,2022-01-21,2022-01-27,10,Percentage,210


In [15]:
promotion_data.sort_values(by = 'PromotionStartDate', inplace=  True)
promotion_data.head()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
162,1066570,2021-10-01,2021-10-07,15,Percentage,45
209,267478,2021-10-01,2021-10-07,10,Percentage,130
85,1006009,2021-10-01,2021-10-07,10,Percentage,70
149,39436,2021-10-01,2021-10-07,25,Percentage,70
151,132334,2021-10-01,2021-10-07,10,Percentage,190


In [29]:
promotion_data.tail()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,SellingPrice,DiscountedPrice
113,1066570,2022-03-11,2022-03-17,0.15,45,38.25
154,1064572,2022-03-11,2022-03-17,0.10,370,333.00
27,999403,2022-03-11,2022-03-17,0.10,260,234.00
311,1076938,2022-03-11,2022-03-17,0.10,80,72.00
136,20824,2022-03-11,2022-03-17,0.10,80,72.00


In [13]:
promotion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ItemCode            314 non-null    int64         
 1   PromotionStartDate  314 non-null    datetime64[ns]
 2   PromotionEndDate    314 non-null    datetime64[ns]
 3   DiscountValue       314 non-null    int64         
 4   DiscountType        314 non-null    object        
 5   SellingPrice        314 non-null    int64         
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 14.8+ KB


In [19]:
promotion_data['DiscountType'].value_counts()

Percentage    311
Amount          3
Name: DiscountType, dtype: int64

In [20]:
promotion_data[promotion_data['DiscountType']=='Amount']

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
35,1082743,2022-01-28,2022-02-03,20,Amount,300
69,1082743,2022-02-04,2022-02-10,20,Amount,300
161,1082743,2022-02-11,2022-02-17,20,Amount,300


In [22]:
amount_idx = promotion_data[promotion_data['DiscountType']=='Amount'].index
amount_idx

Int64Index([35, 69, 161], dtype='int64')

In [23]:
p_data = promotion_data.copy()

In [24]:
for i in amount_idx:
  promotion_data['DiscountValue'].iloc[i] = (promotion_data['DiscountValue'].iloc[i]/ promotion_data['SellingPrice'].iloc[i])*100
promotion_data[promotion_data['DiscountType']=='Amount']

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,DiscountType,SellingPrice
35,1082743,2022-01-28,2022-02-03,20.0,Amount,300
69,1082743,2022-02-04,2022-02-10,20.0,Amount,300
161,1082743,2022-02-11,2022-02-17,20.0,Amount,300


In [26]:
promotion_data.drop(['DiscountType'], axis=1,inplace = True)
promotion_data['DiscountValue'] = promotion_data['DiscountValue']/100
promotion_data['DiscountedPrice'] = promotion_data['SellingPrice']*(1-promotion_data['DiscountValue'])
promotion_data.head()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,SellingPrice,DiscountedPrice
162,1066570,2021-10-01,2021-10-07,0.15,45,38.25
209,267478,2021-10-01,2021-10-07,0.10,130,117.00
85,1006009,2021-10-01,2021-10-07,0.10,70,63.00
149,39436,2021-10-01,2021-10-07,0.25,70,52.50
151,132334,2021-10-01,2021-10-07,0.10,190,171.00


In [27]:
promotion_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 162 to 136
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ItemCode            314 non-null    int64         
 1   PromotionStartDate  314 non-null    datetime64[ns]
 2   PromotionEndDate    314 non-null    datetime64[ns]
 3   DiscountValue       314 non-null    float64       
 4   SellingPrice        314 non-null    int64         
 5   DiscountedPrice     314 non-null    float64       
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 25.3 KB


In [32]:
promotion_data['WeekID'] = [week_id(i) for i in promotion_data['PromotionStartDate']]
promotion_data.head()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,SellingPrice,DiscountedPrice,WeekID
162,1066570,2021-10-01,2021-10-07,0.15,45,38.25,1
209,267478,2021-10-01,2021-10-07,0.10,130,117.00,1
85,1006009,2021-10-01,2021-10-07,0.10,70,63.00,1
149,39436,2021-10-01,2021-10-07,0.25,70,52.50,1
151,132334,2021-10-01,2021-10-07,0.10,190,171.00,1


In [33]:
promotion_data.tail()

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,SellingPrice,DiscountedPrice,WeekID
113,1066570,2022-03-11,2022-03-17,0.15,45,38.25,24
154,1064572,2022-03-11,2022-03-17,0.10,370,333.00,24
27,999403,2022-03-11,2022-03-17,0.10,260,234.00,24
311,1076938,2022-03-11,2022-03-17,0.10,80,72.00,24
136,20824,2022-03-11,2022-03-17,0.10,80,72.00,24


In [46]:
promotion_data['OnPromotion'] = 1.0
promotion_data

,ItemCode,PromotionStartDate,PromotionEndDate,DiscountValue,SellingPrice,DiscountedPrice,WeekID,OnPromotion
162,1066570,2021-10-01,2021-10-07,0.15,45,38.25,1,1.0
209,267478,2021-10-01,2021-10-07,0.10,130,117.00,1,1.0
85,1006009,2021-10-01,2021-10-07,0.10,70,63.00,1,1.0
149,39436,2021-10-01,2021-10-07,0.25,70,52.50,1,1.0
151,132334,2021-10-01,2021-10-07,0.10,190,171.00,1,1.0
...,...,...,...,...,...,...,...,...
113,1066570,2022-03-11,2022-03-17,0.15,45,38.25,24,1.0
154,1064572,2022-03-11,2022-03-17,0.10,370,333.00,24,1.0
27,999403,2022-03-11,2022-03-17,0.10,260,234.00,24,1.0
311,1076938,2022-03-11,2022-03-17,0.10,80,72.00,24,1.0


## Train data

In [10]:
train_data.head()

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,1071115,2021-12-19,1
1,category_4,210652,2022-02-13,1
2,category_2,877624,2021-12-19,1
3,category_1,371104,2021-12-19,3
4,category_1,1090294,2021-12-19,3


In [14]:
train_data.sort_values(by = 'DateID', inplace = True)
train_data.head()

,CategoryCode,ItemCode,DateID,DailySales
6447,category_2,117610,2021-10-01,1
6354,category_4,245581,2021-10-01,6
6355,category_2,1090258,2021-10-01,2
6356,category_2,39436,2021-10-01,60
6357,category_3,1090024,2021-10-01,4


In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20651 entries, 0 to 20650
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CategoryCode  20651 non-null  object        
 1   ItemCode      20651 non-null  int64         
 2   DateID        20651 non-null  datetime64[ns]
 3   DailySales    20651 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 645.5+ KB


In [34]:
train_data['Week'] = [week_number(i) for i in train_data['DateID']]
train_data['WeekID'] = [week_id(i) for i in train_data['Week']]
train_data.head()

,CategoryCode,ItemCode,DateID,DailySales,Week,WeekID
6447,category_2,117610,2021-10-01,1,w1,1
6354,category_4,245581,2021-10-01,6,w1,1
6355,category_2,1090258,2021-10-01,2,w1,1
6356,category_2,39436,2021-10-01,60,w1,1
6357,category_3,1090024,2021-10-01,4,w1,1


In [35]:
train_data.tail()

,CategoryCode,ItemCode,DateID,DailySales,Week,WeekID
18195,category_3,16936,2022-02-17,2,w1,21
18199,category_1,371239,2022-02-17,2,w1,21
18200,category_1,1030948,2022-02-17,2,w1,21
18182,category_1,52099,2022-02-17,2,w1,21
17641,category_2,111436,2022-02-17,1,w1,21


In [36]:
t_data = train_data.copy()

In [45]:
train_data = t_data

In [47]:
train_data = pd.merge(train_data, promotion_data[['ItemCode','WeekID','DiscountValue','DiscountedPrice', 'SellingPrice', 'OnPromotion']], on = ['WeekID', 'ItemCode'], how = 'left')
train_data.head()

,CategoryCode,ItemCode,DateID,DailySales,Week,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
0,category_2,117610,2021-10-01,1,w1,1,NaN,NaN,NaN,NaN
1,category_4,245581,2021-10-01,6,w1,1,NaN,NaN,NaN,NaN
2,category_2,1090258,2021-10-01,2,w1,1,NaN,NaN,NaN,NaN
3,category_2,39436,2021-10-01,60,w1,1,0.25,52.5,70.0,1.0
4,category_3,1090024,2021-10-01,4,w1,1,NaN,NaN,NaN,NaN


In [48]:
train_data['OnPromotion'].fillna(0, inplace = True)
train_data

,CategoryCode,ItemCode,DateID,DailySales,Week,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
0,category_2,117610,2021-10-01,1,w1,1,NaN,NaN,NaN,0.0
1,category_4,245581,2021-10-01,6,w1,1,NaN,NaN,NaN,0.0
2,category_2,1090258,2021-10-01,2,w1,1,NaN,NaN,NaN,0.0
3,category_2,39436,2021-10-01,60,w1,1,0.25,52.5,70.0,1.0
4,category_3,1090024,2021-10-01,4,w1,1,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
20646,category_3,16936,2022-02-17,2,w1,21,NaN,NaN,NaN,0.0
20647,category_1,371239,2022-02-17,2,w1,21,NaN,NaN,NaN,0.0
20648,category_1,1030948,2022-02-17,2,w1,21,NaN,NaN,NaN,0.0
20649,category_1,52099,2022-02-17,2,w1,21,NaN,NaN,NaN,0.0


In [49]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20651 entries, 0 to 20650
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CategoryCode     20651 non-null  object        
 1   ItemCode         20651 non-null  int64         
 2   DateID           20651 non-null  datetime64[ns]
 3   DailySales       20651 non-null  int64         
 4   Week             20651 non-null  object        
 5   WeekID           20651 non-null  int64         
 6   DiscountValue    1585 non-null   float64       
 7   DiscountedPrice  1585 non-null   float64       
 8   SellingPrice     1585 non-null   float64       
 9   OnPromotion      20651 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 1.7+ MB


In [74]:
train_data.to_csv('train_data_modified.csv', index = False)

## Validation data

In [69]:
valid_data['WeekID'] = [week_id_valid(i)+20 for i in valid_data['Week']]
valid_data = pd.merge(valid_data, promotion_data[['ItemCode','WeekID','DiscountValue','DiscountedPrice', 'SellingPrice', 'OnPromotion']], on = ['WeekID', 'ItemCode'], how = 'left')
valid_data['OnPromotion'].fillna(0, inplace = True)
valid_data.head()

,CategoryCode,ItemCode,Week,OnPromo,WeeklySales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
0,category_2,1006090,w1,0,9,21,NaN,NaN,NaN,0.0
81,category_1,1085749,w1,1,17,21,0.1,63.0,70.0,1.0
170,category_1,1090105,w1,0,9,21,NaN,NaN,NaN,0.0
312,category_2,836125,w1,0,34,21,NaN,NaN,NaN,0.0
255,category_1,1090294,w1,0,24,21,NaN,NaN,NaN,0.0


In [70]:
valid_data.tail()

,CategoryCode,ItemCode,Week,OnPromo,WeeklySales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
140,category_3,1066570,w4,1,41,24,0.15,38.25,45.0,1.0
272,category_2,1090276,w4,0,8,24,NaN,NaN,NaN,0.0
271,category_1,36898,w4,0,142,24,NaN,NaN,NaN,0.0
120,category_1,1050046,w4,0,62,24,NaN,NaN,NaN,0.0
372,category_1,3418,w4,0,77,24,NaN,NaN,NaN,0.0


In [75]:
valid_data.to_csv('validation_data_modified.csv', index = False)

## Test Data

In [72]:
test_data['WeekID'] = [week_id_valid(i)+20 for i in test_data['Week']]
test_data.sort_values(by='WeekID', inplace = True)
test_data = pd.merge(test_data, promotion_data[['ItemCode','WeekID','DiscountValue','DiscountedPrice', 'SellingPrice', 'OnPromotion']], on = ['WeekID', 'ItemCode'], how = 'left')
test_data['OnPromotion'].fillna(0, inplace = True)
test_data.head()

,CategoryCode,ItemCode,Week,OnPromo,PredictedSales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
0,category_2,32245,w1,0,NaN,21,NaN,NaN,NaN,0.0
1,category_1,1056463,w1,0,NaN,21,NaN,NaN,NaN,0.0
2,category_1,119554,w1,0,NaN,21,NaN,NaN,NaN,0.0
3,category_2,815101,w1,0,NaN,21,NaN,NaN,NaN,0.0
4,category_1,1054978,w1,0,NaN,21,NaN,NaN,NaN,0.0


In [73]:
test_data.tail()

,CategoryCode,ItemCode,Week,OnPromo,PredictedSales,WeekID,DiscountValue,DiscountedPrice,SellingPrice,OnPromotion
372,category_1,145330,w4,0,NaN,24,NaN,NaN,NaN,0.0
373,category_1,1032568,w4,0,NaN,24,NaN,NaN,NaN,0.0
374,category_2,838456,w4,0,NaN,24,NaN,NaN,NaN,0.0
375,category_4,1003192,w4,0,NaN,24,NaN,NaN,NaN,0.0
376,category_1,38518,w4,0,NaN,24,NaN,NaN,NaN,0.0


In [76]:
test_data.to_csv('test_data_modified.csv', index = False)